In [5]:
import json
import jsonlines

def translate_medical_notes(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Create the output file in jsonl format
    with jsonlines.open(output_file, mode='w') as writer:
        count = 0  # Track the number of entries written
        
        for entry in data:
            output_entry = {
                "Original clinical note": entry['original_english'],
                "translation": entry['translated_spanish']
            }
            writer.write(output_entry)
            
            count += 1
            
            # Stop after writing 3 entries
            if count == 3:
                break

# Example usage
translate_medical_notes("/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct.json", 
                        "/project/pi_hongyu_umass_edu/zonghai/hospital_translation/self-refine/data/tasks/ML/translated_inital.jsonl")


In [ ]:
# Run the code to generate fed_data.jsonl:
import json
import jsonlines
def translate_medical_notes(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    output_data = []
    for entry in data:
        output_entry = {
            "Original clinical note": entry['english'],
            # "translation": "",
            # "annotations": {
                
            #     # "Clinical usefulness":[],
            #     # "Clinical Accuracy":[],"
            #     # "Overall Clarity":[],
            #     # "Coverage":[],
            #     # "Fluency":[],
            #     # "total_score"
            # }
        }
        output_data.append(output_entry)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)

# Example usage
translate_medical_notes("/project/pi_hongyu_umass_edu/zonghai/hospital_translation/self-refine/data/Sampled_100_MedlinePlus_eng_spanish_pair.json", "/project/pi_hongyu_umass_edu/zonghai/hospital_translation/self-refine/data/tasks/ML/fed_data.json")


In [5]:
import json
import re

# Define the input and output file paths
input_filepath = "/project/pi_hongyu_umass_edu/zonghai/hospital_translation/self-refine/trans_gpt5.json"
output_filepath = '/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/gpt4o_self_refine_trans_5.json'
original_spa_filepath = '/project/pi_hongyu_umass_edu/zonghai/hospital_translation/self-refine/data/Sampled_100_MedlinePlus_eng_spanish_pair.json'

# Function to remove the score part and anything after it from the translation

def remove_score_and_after(translation):
    return re.sub(r'\s*\[score:.*$', '', translation, flags=re.DOTALL).strip()


# Read the original Spanish translations JSON file
with open(original_spa_filepath, 'r', encoding='utf-8') as infile:
    original_spa_data = {entry["english"]: entry["spanish"] for entry in json.load(infile)}

# Read the input JSON file
with open(input_filepath, 'r', encoding='utf-8') as infile:
    data = [json.loads(line) for line in infile]

# Process the data to extract the required fields
cleaned_data = []
for entry in data:
    original_eng = entry.get("Original clinical note", "")
    generated_translation = entry.get("generated_translations", "")
    cleaned_translation = remove_score_and_after(generated_translation)
    original_spa = original_spa_data.get(original_eng, "")
    cleaned_data.append({
        "original_eng": original_eng,
        "original_spa": original_spa,
        "tran_spa": cleaned_translation
    })

# Write the cleaned data to the output JSON file
with open(output_filepath, 'w', encoding='utf-8') as outfile:
    json.dump(cleaned_data, outfile, ensure_ascii=False, indent=4)

print(f"Cleaned data has been written to {output_filepath}")

Cleaned data has been written to /project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/gpt4o_self_refine_trans_5.json
